# Advent of Code 2022

I liked [Peter Norvig's approach](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb) [last year](https://github.com/codemonkeyjim/adventofcode-2021/blob/main/aoc-2021.ipynb), so I'm going to use it again this year.

## Day 0: Imports and Utility Functions
Preparations prior to Day 1:

- Some imports.
- A way to read the day's data file and to print/check the output.
- Some utilities that are likely to be useful.


In [ ]:
from __future__ import annotations
from anytree import NodeMixin, RenderTree
import anytree.search as treesearch
from collections import Counter, defaultdict, namedtuple
from copy import deepcopy
from dataclasses import dataclass
from functools import reduce
from itertools import accumulate, chain, islice, permutations, takewhile, zip_longest
from math import copysign, prod
import numpy as np
import operator
from parse import parse
from pprint import pprint
from queue import PriorityQueue
from statistics import mean, median
from typing import Callable


In [ ]:
def data(day: int, parser=str, sep='\n', filetype="input") -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'data/advent2022/{filetype}{day}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]
     
def do(day, *answers) -> dict[int, int]:
    "E.g., do(3) returns {1: day3_1(in3), 2: day3_2(in3)}. Verifies `answers` if given."
    g = globals()
    got = []
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g: 
            got.append(g[fname](g[f'in{day}']))
            if len(answers) >= part: 
                assert got[-1] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[-1]}; expected {answers[part - 1]}')
    return got

def by_line(text: str) -> list[str]:
    "Split the text into a list of lines."
    return text.strip().splitlines()

def first(iterable, default=None) -> object:
    "Return first item in iterable, or default."
    return next(iter(iterable), default)

def rest(sequence) -> object: return sequence[1:]

def take_n(iterable, n=1, fillvalue = None):
  slices = (islice(iterable, i, None, n) for i in range(n))
  return zip_longest(*slices, fillvalue = fillvalue)

## Day 1: Calorie Counting

1. Find the Elf carrying the most Calories. How many total Calories is that Elf carrying?
2. Find the top three Elves carrying the most Calories. How many Calories are those Elves carrying in total?

In [ ]:
in1 = data(1, parser=lambda lines: [int(line) for line in lines.split("\n")], sep="\n\n")

In [ ]:
def day1_1(packs: list(int)) -> int:
    return max([sum(pack) for pack in packs])

In [ ]:
def day1_2(packs: list(int)) -> int:
    return sum(sorted([sum(pack) for pack in packs])[-3:])

In [ ]:
do(1, 69626)

## Day 2: Rock Paper Scissors

1. What would your total score be if everything goes exactly according to your strategy guide?

In [ ]:
in2 = data(2, parser=lambda line: line.split(" "))

In [ ]:
OPPONENT_MAP = {
    'A': 0,
    'B': 1,
    'C': 2,
}

MY_MAP = {
    'X': 0,
    'Y': 1,
    'Z': 2,
}

OUTCOME_SCORE = [
    3, # Tie
    6, # Win
    0, # Loss
]

Hand = list[str]

In [ ]:
def hand_score(hand: Hand, opponent_map:dict[str, int]=OPPONENT_MAP, my_map: dict[str, int]=MY_MAP) -> int:
    opponent_play = opponent_map[hand[0]]
    my_play = my_map[hand[1]]

    round_score = OUTCOME_SCORE[(my_play - opponent_play) % 3]
    return round_score + my_play + 1

assert hand_score(['A', 'Y']) == 8
assert hand_score(['B', 'X']) == 1
assert hand_score(['C', 'Z']) == 6


In [ ]:
def day2_1(hands: list[Hand]) -> int:
    return sum([hand_score(hand) for hand in hands])

Don't get fancy. The number of options is small enough to hardcode a table of hands: what to play to get the specified outcome.

In [ ]:
FIX_HAND = {
    'A': {'X': 'Z', 'Y': 'X', 'Z': 'Y'},
    'B': {'X': 'X', 'Y': 'Y', 'Z': 'Z'},
    'C': {'X': 'Y', 'Y': 'Z', 'Z': 'X'},
}
def fix_hand(hand: Hand) -> Hand:
    return [hand[0], FIX_HAND[hand[0]][hand[1]]]

In [ ]:
def day2_2(hands: list[Hand]) -> int:
    return day2_1([fix_hand(hand) for hand in hands])

In [ ]:
do(2, 11603)

## Day 3: Rucksack Reorganization

1. Find the item type that appears in both compartments of each rucksack. What is the sum of the priorities of those item types?

In [ ]:
Sack = tuple[set[str], set[str]]

def compartmentalize(sack: str) -> Sack:
    mid = len(sack) // 2
    return (set(sack[:mid]), set(sack[mid:]))

assert compartmentalize("abcdefgh") == ({'a', 'b', 'c', 'd'}, {'e', 'f', 'g', 'h'})

def both_sides(sack: Sack) -> str:
    items = list(sack[0].intersection(sack[1]))
    assert len(items) == 1
    return items[0]

def whole_sack(sack: Sack) -> set[str]:
    return sack[0].union(sack[1])

def common_item(sacks: list[Sack]) -> str:
    items = list(reduce(lambda intersection, sack: intersection.intersection(sack), sacks))
    assert len(items) == 1
    return items[0]

item_priorities = {**{chr(val): val - ord('a') + 1 for val in range(ord('a'), ord('z')+1)}, **{chr(val): val - ord('A') + 27 for val in range(ord('A'), ord('Z')+1)}}

In [ ]:
in3 = data(3, parser=compartmentalize)

In [ ]:
def day3_1(sacks: list[Sack]) -> int:
    return sum([item_priorities[both_sides(sack)] for sack in sacks])

In [ ]:
def day3_2(sacks: list[Sack]) -> int:
    return sum([item_priorities[common_item(list(map(whole_sack, elf_group)))] for elf_group in take_n(sacks, 3)])


In [ ]:
do(3, 7848, 2616)

## Day 4: Camp Cleanup

1. In how many assignment pairs does one range fully contain the other?

In [ ]:
@dataclass
class Sections:
    start: int
    end: int

    def contains(self, other: Sections) -> bool:
        return self.start <= other.start and self.end >= other.end

    def overlaps(self, other: Sections) -> bool:
        overlap = Sections(max(self.start, other.start), min(self.end, other.end))
        return overlap.start <= overlap.end

@dataclass
class AssignmentPair:
    left: Sections
    right: Sections

def parse_cleanup_pairs(line: str) -> AssignmentPair:
    result = parse("{:d}-{:d},{:d}-{:d}", line)
    return AssignmentPair(Sections(*result[0:2]), Sections(*result[2:4]))

In [ ]:
in4 = data(4, parser=parse_cleanup_pairs)

In [ ]:
def day4_1(assignments: list[AssignmentPair]) -> int:
    return sum([pair.left.contains(pair.right) or pair.right.contains(pair.left) for pair in assignments])

In [ ]:
def day4_2(assignments: list[AssignmentPair]) -> int:
    return sum([pair.left.overlaps(pair.right) for pair in assignments])

In [ ]:
do(4, 305)

## Day 5: Supply Stacks

1. After the rearrangement procedure completes, what crate ends up on top of each stack? (move one at a time)
2. After the rearrangement procedure completes, what crate ends up on top of each stack? (move all at once)

In [ ]:
Crate = str
Stack = list[Crate]

@dataclass
class Move:
    n: int
    start: int
    end: int

def parse_stack_lines(lines: str, num_stacks: int = 9) -> list[Stack]:
    stacks = []
    stack_values = range(1, 4*num_stacks, 4)

    for _ in range(num_stacks+1):
        stacks.append(Stack())

    for line in lines.split("\n"):
        if line == '' or line.startswith(' 1'):
            continue
        else:
            # Stack numbers are 1-indexed
            for stack, position in enumerate(stack_values, start=1):
                crate = line[position]
                if crate != ' ':
                    # Stacks are built upside down
                    stacks[stack].append(crate)
    
    return [list(reversed(stack)) for stack in stacks]

def parse_move_lines(lines: str) -> list[Move]:
    moves = []
    for line in lines.split("\n"):
        move = Move(*parse("move {:d} from {:d} to {:d}", line))
        moves.append(move)
    return moves


In [ ]:
stack_lines, move_lines = data(5, sep="\n\n")
in5 = (parse_stack_lines(stack_lines), parse_move_lines(move_lines))
# stack_lines, move_lines = data(5, sep="\n\n", filetype="sample")
# in5 = (parse_stack_lines(stack_lines, num_stacks=3), parse_move_lines(move_lines))

In [ ]:
def day5_1(inputs: tuple(list[Stack], list[Move])) -> str:
    stacks = deepcopy(inputs[0])
    moves = inputs[1]
    
    for move in moves:
        for _ in range(move.n):
            stacks[move.end].append(stacks[move.start].pop())
    return ''.join([stack[-1] for stack in stacks if len(stack) > 0])

In [ ]:
def day5_2(inputs: tuple(list[Stack], list[Move])) -> str:
    stacks = deepcopy(inputs[0])
    moves = inputs[1]
    
    for move in moves:
        stacks[move.end].extend(stacks[move.start][-move.n:])
        stacks[move.start] = stacks[move.start][:-move.n]
    return ''.join([stack[-1] for stack in stacks if len(stack) > 0])

In [ ]:
do(5, 'RTGWZTHLD', 'STHGRZZFR')

## Day 6: Tuning Trouble

1. How many characters need to be processed before the first start-of-packet marker is detected?
2. How many characters need to be processed before the first start-of-message marker is detected?

In [ ]:
def start_of_packet(msg: str, marker_len: int = 4) -> int:
    for start in range(len(msg)):
        end = start + marker_len
        marker = msg[start:end]
        if len(set(marker)) == marker_len:
            return end
    return None

assert start_of_packet("mjqjpqmgbljsphdztnvjfqwrcgsmlb") == 7

In [ ]:
in6 = data(6, sep = "\n\n")[0]

In [ ]:
def day6_1(msg: str) -> int:
    return start_of_packet(msg)

In [ ]:
def day6_2(msg: str) -> int:
    return start_of_packet(msg, marker_len=14)

In [ ]:
do(6, 1198, 3120)

## Day 7: No Space Left On Device

1. Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?
2. Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?

In [ ]:
class DirNode(NodeMixin):
    def __init__(self, name, parent: DirNode=None, children=list()):
        super(DirNode, self).__init__()
        self.name = name
        self.parent = parent
        self.children = children
        self.size = 0
    
    def get_or_make_child(self, child_name, size=0):
        is_dir = size == 0
        candidates = [child for child in self.children if child.name == child_name]
        match len(candidates):
            case 0:
                return DirNode(child_name, parent=self) if is_dir else FileNode(child_name, size=size, parent=self)
            case 1:
                return candidates[0]
            case _:
                raise Exception(f'Current {self.name} has too many children matching {child_name}: {candidates}')
    
    def increase_size(self, size: int):
        self.size += size
        if self.parent is not None:
            self.parent.increase_size(size)
    
    def __str__(self):
        return f'{self.name} (dir)'

class FileNode(NodeMixin):
    def __init__(self, name, size, parent: DirNode=None):
        super(FileNode, self).__init__()
        self.name = name
        self.parent = parent
        self.children = list()
        self.size = size
        self.parent.increase_size(size)
    
    def __str__(self):
        return f'{self.name} (file, size={self.size})'


def parse_listings(lines: list[str]) -> DirNode:
    root = DirNode('')
    curr = root

    for line in lines:
        match line.split(' '):
            case ['$', 'cd', '/']:
                curr = root
            case ['$', 'cd', '..']:
                curr = curr.parent
            case ['$', 'cd', dir_name]:
                curr = curr.get_or_make_child(dir_name)
            case ['$', 'ls']:
                pass
            case ['dir', dir_name]:
                curr.get_or_make_child(dir_name)
            case [size, filename]:
                curr.get_or_make_child(filename, int(size))
    return root

In [ ]:
root = parse_listings(data(7, filetype='sample'))
print(RenderTree(root).by_attr(lambda n: str(n)))


In [ ]:
in7 = parse_listings(data(7))

In [ ]:
def day7_1(root: DirNode) -> int:
    return sum([node.size for node in treesearch.findall(root, filter_=lambda n: type(n) == DirNode and n.size <= 100_000)])

In [ ]:
def day7_2(root: DirNode) -> int:
    DISK_SIZE = 70_000_000
    MIN_FREE = 30_000_000
    target = MIN_FREE - (DISK_SIZE - root.size)

    return min([node.size for node in treesearch.findall(root, filter_=lambda n: type(n) == DirNode and n.size >= target)])

In [ ]:
do(7, 1086293, 366028)

## Day 8: Treetop Tree House

1. How many trees are visible from outside the grid?
2. Consider each tree on your map. What is the highest scenic score possible for any tree?

In [ ]:
sample8 = np.array(data(8, parser=lambda line: [int(i) for i in line], filetype="sample"))
in8 = np.array(data(8, parser=lambda line: [int(i) for i in line]))

In [ ]:
def is_increasing(row: np.array) -> np.array:
    prev_max = 0
    prev_result = True

    it = np.nditer([row, None])
    with it:
        for val, incr in it:
            prev_result = val > prev_max
            prev_max = max(val, prev_max)
            incr[...] = prev_result
        return it.operands[1]

In [ ]:
def visible_trees(forest: np.ndarray) -> np.ndarray:
    lr = np.apply_along_axis(is_increasing, 1, forest)
    tb = np.apply_along_axis(is_increasing, 0, forest)
    rl = np.fliplr(np.apply_along_axis(is_increasing, 1, np.fliplr(forest)))
    bt = np.flipud(np.apply_along_axis(is_increasing, 0, np.flipud(forest)))

    border = np.zeros_like(forest)
    border[0:1] = 1
    border[-1:] = 1
    border[:,0] = 1
    border[:,-1] = 1

    return lr | tb | rl | bt | border

In [ ]:
def day8_1(forest: np.ndarray) -> int:
    return np.sum(visible_trees(forest))

For any tree `(i,j)` in the `forest`, generate a list containing lists of the trees in each of the four directions from `(i,j)` (but not including it).

In [ ]:
def tree_views(forest: np.ndarray, i: int, j: int):
    indices = [
        (range(i, -1, -1), j),  # up
        (i, range(j, -1, -1)),  # left
        (range(i, forest.shape[0]), j),  # down
        (i, range(j, forest.shape[1])),  # right
    ]
    return [forest[idx][1:] for idx in indices]

def view_score(view: np.array, height: int) -> int:
    score = len(list(takewhile(lambda x: x < height, view)))
    # Include the tree we stopped at if it's not the edge
    if score < len(view):
        score += 1
    return score

In [ ]:
def day8_2(forest: np.ndarray) -> int:
    best = 0
    for i in range(forest.shape[0]):
        for j in range(forest.shape[1]):
            score = reduce(operator.mul, [view_score(view, forest[i][j]) for view in tree_views(forest, i, j)], 1)
            if score > best:
                best = score
    return best

In [ ]:
do(8, 1700, 470596)

## Day 9: Rope Bridge

1. How many positions does the tail of the rope visit at least once?


In [ ]:
def expand_directions(line: str) -> list[str]:
    dir, n = line.split(' ', maxsplit=2)
    return [dir] * int(n)

in9 = list(chain(*data(9, parser=expand_directions)))

In [ ]:
@dataclass
class Point:
    x: int
    y: int

    @property
    def is_diag(self):
        return abs(self.x) != 0 and abs(self.y) != 0
    
    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y)
    
    def __sub__(self, other):
        return Point(self.x - other.x, self.y - other.y)
    
    def __hash__(self):
        return hash(tuple([self.x, self.y]))

In [ ]:
def print_rope(rope: list[Point]) -> None:
    xs = [p.x for p in rope]
    min_x = min(xs)
    max_x = max(xs)

    ys = [p.y for p in rope]
    min_y = min(ys)
    max_y = max(ys)

    plane = list()
    for y in range(max_y - min_y + 1):
        plane.append(['.'] * (max_x - min_x + 1))
    plane[0][0] = 's'

    for i, pt in enumerate(rope):
        plane[pt.y - min_y][pt.x - min_x] = str(i)
    
    for row in reversed(plane):
        print(''.join(row))

In [ ]:
def move_knot(head: Point, tail: Point) -> Point:
    diff = head - tail
    if abs(diff.x) <= 1 and abs(diff.y) <= 1:
        return Point(tail.x, tail.y)
    
    move = Point(0, 0)

    if abs(diff.x) > 1 or diff.is_diag:
        move.x = int(copysign(1, diff.x))
    if abs(diff.y) > 1 or diff.is_diag:
        move.y = int(copysign(1, diff.y))
    
    return move + tail

STEPS = {
    'L': Point(-1,  0),
    'R': Point( 1,  0),
    'U': Point( 0,  1),
    'D': Point( 0, -1),
}

def simulate_rope(steps: list[str], rope_length: int) -> int:
    rope = [Point(0,0)] * rope_length
    visited = set([rope[-1]])

    for step in steps:
        rope[0] = rope[0] + STEPS[step]
        for i in range(rope_length-1):
            # print(f'{i+1}: {rope[i]}, {rope[i+1]}')
            rope[i+1] = move_knot(rope[i], rope[i+1])
        visited.add(rope[-1])
        # print_rope(rope)
        # print()
    return len(visited)


In [ ]:
def day9_1(steps: list[str]) -> int:
    return simulate_rope(steps, rope_length=2)

In [ ]:
def day9_2(steps: list[str]) -> int:
    return simulate_rope(steps, rope_length=10)

In [ ]:
sample9 = list(chain(*data(9, parser=expand_directions, filetype="sample")))
day9_2(sample9)

In [ ]:
do(9, 6181, 2386)